# Fonctions avancées d'Interface

Installez les bibliothèques 🤗 Transformers et 🤗 Gradio pour exécuter ce *notebook*.

In [ ]:
!pip install datasets transformers[sentencepiece]
!pip install gradio

In [ ]:
import random

import gradio as gr


def chat(message, history):
    history = history or []
    if message.startswith("Combien"):
        response = random.randint(1, 10)
    elif message.startswith("Comment"):
        response = random.choice(["Super", "Bon", "Ok", "Mal"])
    elif message.startswith("Où"):
        response = random.choice(["Ici", "Là", "Quelque part"])
    else:
        response = "Je ne sais pas."
    history.append((message, response))
    return history, history


iface = gr.Interface(
    chat,
    ["text", "state"],
    ["chatbot", "state"],
    allow_screenshot=False,
    allow_flagging="never",
)
iface.launch()

In [ ]:
import requests
import tensorflow as tf

import gradio as gr

inception_net = tf.keras.applications.MobileNetV2()  # charger le modèle

# Télécharger des étiquettes lisibles par l'homme pour ImageNet
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")


def classify_image(inp):
    inp = inp.reshape((-1, 224, 224, 3))
    inp = tf.keras.applications.mobilenet_v2.preprocess_input(inp)
    prediction = inception_net.predict(inp).flatten()
    return {labels[i]: float(prediction[i]) for i in range(1000)}


image = gr.Image(shape=(224, 224))
label = gr.Label(num_top_classes=3)

title = "Classification des images avec Gradio + Exemple d'interprétation"
gr.Interface(
    fn=classify_image, inputs=image, outputs=label, interpretation="default", title=title
).launch()